Clean layer (SQL view) for tip analysis

In order to analyze tipping behavior consistently, a clean layer was created as an SQL view (vw_yellow_clean_tip). The view encapsulates basic plausibility rules (e.g., drop-off after pickup, positive fare and distance, non-negative tip values) and calculates analysis features such as trip duration in minutes, pickup hour, day of the week, and the KPIs is_tipped and tip_rate.

In addition, pickup and drop-off zones are joined via the dim_taxi_zone dimension to evaluate geographical patterns in tipping behavior (hotspots, boroughs, airports). By implementing this as a view, the raw data table remains unchanged, while the analysis remains reproducible and easily adaptable.

In [5]:
##Set Up für jedes Notebook

import os
from dotenv import load_dotenv
from urllib.parse import quote_plus
from sqlalchemy import create_engine, text

load_dotenv()

PG_USER = os.getenv("POSTGRES_USER")
PG_PASS = quote_plus(os.getenv("POSTGRES_PASS"))
PG_HOST = os.getenv("POSTGRES_HOST")
PG_PORT = os.getenv("POSTGRES_PORT", "5432")
PG_DB   = os.getenv("POSTGRES_DB")
PG_SCHEMA = os.getenv("POSTGRES_SCHEMA", "public")

missing = [k for k,v in {
    "POSTGRES_USER": PG_USER,
    "POSTGRES_PASS": os.getenv("POSTGRES_PASS"),
    "POSTGRES_HOST": PG_HOST,
    "POSTGRES_DB": PG_DB,
    "POSTGRES_SCHEMA": PG_SCHEMA
}.items() if not v]
if missing:
    raise ValueError(f"Missing env vars: {missing}")

url = f"postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"
engine = create_engine(url, future=True)

# kurzer Ping
with engine.connect() as conn:
    print(conn.execute(text("SELECT current_user, current_database(), current_schema();")).fetchone())

print("Setup OK. Schema:", PG_SCHEMA)


('patrickpaubandt', 'nf_da_onl_13102025', 'public')
Setup OK. Schema: s_patrickpaubandt


In [ ]:
from sqlalchemy import text

sql = f"""
CREATE OR REPLACE VIEW {PG_SCHEMA}.vw_yellow_clean_tip AS
SELECT
    -- the existing data fields (already created in the staging table)
    t.vendorid,
    t.tpep_pickup_datetime,
    t.tpep_dropoff_datetime,
    t.passenger_count,
    t.trip_distance,
    t.ratecodeid,
    t.store_and_fwd_flag,
    t.pulocationid,
    t.dolocationid,
    t.payment_type,
    t.fare_amount,
    t.extra,
    t.mta_tax,
    t.tip_amount,
    t.tolls_amount,
    t.improvement_surcharge,
    t.total_amount,
    t.congestion_surcharge,
    t.airport_fee,
    t.cbd_congestion_fee,
    t.source_file,
    t.ingested_at,

    -- Time features (for later grouping)
    EXTRACT(HOUR FROM t.tpep_pickup_datetime)::int AS pickup_hour,
    EXTRACT(DOW  FROM t.tpep_pickup_datetime)::int AS pickup_dow,   -- 0=Sun ... 6=Sat
    DATE_TRUNC('day', t.tpep_pickup_datetime)::date AS pickup_date,
    DATE_TRUNC('month', t.tpep_pickup_datetime)::date AS pickup_month,

    -- Duration in minutes (important: tip may correlate with trip duration)
    EXTRACT(EPOCH FROM (t.tpep_dropoff_datetime - t.tpep_pickup_datetime)) / 60.0 AS duration_min,

    -- Tip KPIs
    (t.tip_amount > 0)::int AS is_tipped,
    CASE WHEN t.fare_amount > 0 THEN (t.tip_amount / t.fare_amount) ELSE NULL END AS tip_rate,

    -- Payment Category
    CASE
      WHEN t.payment_type = 1 THEN 'Card'
      WHEN t.payment_type = 2 THEN 'Cash'
      ELSE 'Other/Unknown'
    END AS payment_group,

    -- Geo Join (Pickup)
    pu.borough AS pu_borough,
    pu.zone AS pu_zone,
    pu.service_zone AS pu_service_zone,

    -- Geo Join (Dropoff)
    do_.borough AS do_borough,
    do_.zone AS do_zone,
    do_.service_zone AS do_service_zone

FROM {PG_SCHEMA}.stg_yellow_trips t
LEFT JOIN {PG_SCHEMA}.dim_taxi_zone pu
  ON t.pulocationid = pu.locationid
LEFT JOIN {PG_SCHEMA}.dim_taxi_zone do_
  ON t.dolocationid = do_.locationid
WHERE
    -- Security mechanisms to exclude incorrect data
    t.tpep_pickup_datetime IS NOT NULL
    AND t.tpep_dropoff_datetime IS NOT NULL
    AND t.tpep_dropoff_datetime > t.tpep_pickup_datetime
    AND t.fare_amount > 0
    AND t.trip_distance > 0
    AND t.tip_amount >= 0;
"""

with engine.begin() as conn:
    conn.execute(text(sql))

print("Created view:", f"{PG_SCHEMA}.vw_yellow_clean_tip")


Created view: s_patrickpaubandt.vw_yellow_clean_tip


In [7]:
## Test: Counts and Tipping Basics

from sqlalchemy import text

with engine.connect() as conn:
    stats = conn.execute(text(f"""
        SELECT
          COUNT(*) AS n_clean,
          AVG(tip_amount) AS avg_tip,
          AVG(fare_amount) AS avg_fare,
          AVG(tip_rate) AS avg_tip_rate,
          AVG(is_tipped) AS share_tipped
        FROM {PG_SCHEMA}.vw_yellow_clean_tip;
    """)).fetchone()

print(stats)


(93819, Decimal('3.9934887389547959'), Decimal('23.0536835822168218'), Decimal('0.201363506256169779785740'), Decimal('0.76921519095279207836'))


In [8]:
##Payment Split

with engine.connect() as conn:
    pay = conn.execute(text(f"""
        SELECT payment_group,
               COUNT(*) AS n,
               AVG(is_tipped) AS share_tipped,
               AVG(tip_rate) AS avg_tip_rate
        FROM {PG_SCHEMA}.vw_yellow_clean_tip
        GROUP BY 1
        ORDER BY n DESC;
    """)).fetchall()

pay


[('Card', 78765, Decimal('0.91620643686916777757'), Decimal('0.239842232674126401885602')),
 ('Cash', 12631, Decimal('0E-24'), Decimal('0E-24')),
 ('Other/Unknown', 2423, Decimal('0.00082542302930251754'), Decimal('0.00022671765168242889'))]